# Optimization Project: Optimizing the U of A Ranch Horse Team Schedule

by: Myli Brown, Jordan Shortt, Chance Pickett

## The Problem
We are optimizing the U of A Ranch Horse Team's Schedule.

There are 5 different shows that the team can attend:

*   SHTX Bryan
*   SHTX World
*   NRCHA
*   Bridles and Brains
*   NIRSHA

There are 6 members of the U of A Ranch Horse Team:
*   Jordan
*   Justine
*   Kate
*   Alex
*   Carson
*   Morgan



For each of these shows, there are a certain number of competitors that can attend, and only a certain number of atendees can score points for the team.

Our overall goal is to optimize the team's score throughout the season.
We are also ensuring that we minimize costs while ensuring a high team score.

We have collected data about the predicted scores for each of the team members for each of the events as well as data about the cost of the shows.

  -Our data is all encoded below, so there isn't a need to import a dataset to run this code -

In [ ]:
# Imports
import pyomo.environ as pyo # pyomo

### Indices:

In [ ]:
#  Indices

I = [1, 2, 3, 4, 5] # shows
'''
  1: SHTX Bryan
  2: SHTX World
  3: NRCHA
  4: Bridles and Brains
  5: NIRSHA
'''

J = [1, 2, 3, 4, 5, 6] # team members
'''
  1: Jordan
  2: Justine
  3: Kate
  4: Alex
  5: Carson
  6: Morgan
'''

show_names = {1: "SHTX Bryan",
              2: "SHTX World",
              3: "NRCHA",
              4: "Bridles and Brains",
              5: "NIRSHA"}

rider_names = {1: "Jordan",
               2: "Justine",
               3: "Kate",
               4: "Alex",
               5: "Carson",
               6: "Morgan"}

### Parameters:

In [ ]:
# Parameters
ro = 120  # rent fee for truck
rn =  55 # rent fee for traverse

do = 12 #Miles per gallon for the truck
dn = 27 #Miles per gallon for the traverse

h = {1: 240,  # cost of hotel room per night for each show
     2: 147,
     3: 215,
     4: 215,
     5: 94}

p = {1: 4,   #Number of nights in the hotel for each show
     2: 5,
     3: 2,
     4: 3,
     5: 4}

q = {1: 200,  # entry fee per person for each show
     2: 200,
     3: 100,
     4: 200,
     5: 265}

t = {1: 35,  # stalling fee per horse for each show
     2: 35,
     3: 330,
     4: 0,
     5: 25}

g = 3 #approximate gas price per gallon

l = {1: len(J),  # show limit for each show
     2: len(J),
     3: 3,
     4: 4,
     5: len(J)}

a = {1: 479,  # miles driven for each show
     2: 496,
     3: 361,
     4: 361,
     5: 471}

k = {1: 4,    # number of team members that can earn points for each show
     2: 4,
     3: 3,
     4: 4,
     5: 4}

c = 30000 #Max cost for the season ($5000 funding each member raises * 6 team members)

x = {(1, 1): 289, (1, 2): 281, (1, 3): 291, (1, 4): 273, (1, 5): 273, (1, 6): 281,  # (Show number (i), Team member (j)) : Total points member j scored at show i.
     (2, 1): 289, (2, 2): 281, (2, 3): 291, (2, 4): 273, (2, 5): 273, (2, 6): 281,
     (3, 1): 145, (3, 2): 137, (3, 3): 145, (3, 4): 136, (3, 5): 130, (3, 6): 137,
     (4, 1): 331, (4, 2): 338, (4, 3): 325, (4, 4): 287, (4, 5): 316, (4, 6): 247,
     (5, 1): 289, (5, 2): 281, (5, 3): 291, (5, 4): 273, (5, 5): 273, (5, 6): 281}

# Team member genders (1 for female, 0 for male)
e = {1: 1,  # Jordan (female)
     2: 1,  # Justine (female)
     3: 1,  # Kate (female)
     4: 0,  # Alex (male)
     5: 1,  # Carson (female)
     6: 1}  # Morgan (female)

### Decision Variables:

$$b_{ij} = \text{binary variable for whether team member j's score was counted in show i (Binary)}$$
$$s_{ij} = \text{binary variable for whether team member j's competed in show i (Binary)}$$
$$o_i = \text{number of trucks needed for show i (Integer)}$$
$$n_i = \text{number of traverses needed for show i (Integer)}$$
$$f_i = \text{number of females in show i (Integer)}$$
$$m_i = \text{number of males in show i (Integer)}$$
$$y_i = \text{total number of female hotel rooms to purchase for show i (Integer)}$$
$$w_i = \text{total number of male hotel rooms to purchase for show i (Integer)}$$
$$z_i = \text{total number of hotel rooms to purchase for show i (Integer)}$$

### Define the Model:

In [ ]:
# Define pyo model
model = pyo.ConcreteModel()
model.clear()

# Define index sets for shows and team members
model.I = pyo.Set(initialize=I)  # shows
model.J = pyo.Set(initialize=J)  # team members

# Define parameters
model.ro = pyo.Param(initialize=ro)  # rent fee for truck
model.rn = pyo.Param(initialize=rn)  # rent fee for traverse
model.h = pyo.Param(model.I, initialize=h)  # cost of hotel room per night for each show
model.p = pyo.Param(model.I, initialize=p)  # Number of nights in the hotel for each show
model.q = pyo.Param(model.I, initialize=q)  # entry fee per person for each show
model.t = pyo.Param(model.I, initialize=t)  # stalling fee per horse for each show
model.g = pyo.Param(initialize=g)  # cost of gas per gallon
model.l = pyo.Param(model.I, initialize=l)  # show limit for each show
model.a = pyo.Param(model.I, initialize=a)  # miles driven for each show
model.k = pyo.Param(model.I, initialize=k)  # number of point riders for each show
model.c = pyo.Param(initialize=c)  # Max cost for the season
model.do = pyo.Param(initialize=do)  # Miles per gallon for the truck
model.dn = pyo.Param(initialize=dn)  # Miles per gallon for the traverse
model.x = pyo.Param(model.I, model.J, initialize=x)  # score for each show and team member
model.e = pyo.Param(model.J, initialize = e)

# Decision Variables
model.b = pyo.Var(model.J, model.I, domain=pyo.Binary)  # binary variable for whether a score is included in overall team score
model.s = pyo.Var(model.J, model.I, domain=pyo.Binary)  # binary for whether a team member participated in show i
model.z = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # num of hotel rooms for show i
model.f = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # num of females in show i
model.m = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # num of males in show i
model.n = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # num of traverses for each show
model.o = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # num of trucks for each show
model.y = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # number of rooms for males
model.w = pyo.Var(model.I, domain=pyo.NonNegativeIntegers)  # number of rooms for females

### The Model:

$$\text{Maximize: } \sum_{j = 1}^6\sum_{i=1}^5 x_{ij}b_{ij} \quad \quad \text{(Overall team score for the season)
}$$

In [ ]:
# Objective: Maximize total team score
model.obj = pyo.Objective(expr=sum(model.x[i, j] * model.b[j, i] for i in model.I for j in model.J), sense=pyo.maximize)

### Constraints:
$$\sum_{i=1}^5s_{ij}\geq 2 \quad \forall j = 1, 2, ..., 6 \quad \text{(Minimum show attendance for each team member)}$$

$$f_i + m_i = \sum_{j=1}^6 s_{ij} \quad \forall i = 1, 2, ..., 5 \quad \text{(maximum show limit)}$$

$$f_i + m_i \leq l_i \quad \forall i = 1, 2, ..., 5 \quad \text{(maximum show limit)}$$

$$f_i = \sum_{j=1}^6 s_{ij}*e_j \quad \forall i = 1, 2, ..., 5 \quad \text{(number of female riders)}$$

$$w_i \geq f_i/4  \quad \forall i = 1, 2, ..., 5 \quad \text{(number of women's hotel rooms)}$$

$$m_i = \sum_{j=1}^6 s_{ij}*(1-e_j) \quad \forall i = 1, 2, ..., 5 \quad \text{(number of male riders)}$$

$$y_i \geq m_i /4  \quad \forall i = 1, 2, ..., 5 \quad \text{(number of men's hotel rooms)}$$

$$z_i = w_i + y_i + 1  \quad \forall i = 1, 2, ..., 5\quad \text{(number of total hotel rooms)}$$

$$\sum_{j=1}^6s_{ij} \leq 5o_i  \quad \forall i = 1, 2, ..., 5 \quad \text{(number of trucks to rent for the number of horses)}$$

$$\sum_{j=1}^6s_{ij} \leq 2o_i + 4n_i  \quad \forall i = 1, 2, ..., 5 \quad \text{(number of vehicles to carry the people)}$$

$$\sum_{j=1}^6b_{ij} = k_i  \quad \forall i = 1, 2, ..., 5 \quad \text{(choose top k scores for each event)}$$

$$b_{ij} \leq s_{ij}  \quad \forall i = 1, 2, ..., 5  \quad \forall j = 1, 2, ..., 6 \quad \text{(A score can only be counted if a team member participates)}$$

$$\sum_{i=1}^{5}(h_i*z_i*p_i)+\sum_{i=1}^5(q_i(f_i+m_i))+\sum_{i=1}^5(t_i(f_i+m_i))+\sum_{i=1}^5((ro+(a_i/do))o_i+(rn+(a_i/dn))n_i) \leq c \quad \text{(Total cost)}$$

In [ ]:
# Constraints
# Minimum attendance for each team member
model.min_attendance = pyo.ConstraintList()
for j in model.J:
    model.min_attendance.add(sum(model.s[j, i] for i in model.I) >= 2)

# Maximum show limit
model.max_show_limit = pyo.ConstraintList()
for i in model.I:
    model.max_show_limit.add(model.f[i] + model.m[i] == sum(model.s[j, i] for j in model.J))
    model.max_show_limit.add(model.f[i] + model.m[i] <= model.l[i])

# Count the number of females and males at each show
model.gender_count = pyo.ConstraintList()
for i in model.I:
    # Count females
    female_expr = sum(model.s[j, i] * model.e[j] for j in model.J)
    model.gender_count.add(model.f[i] == female_expr)

    # Count males
    male_expr = sum(model.s[j, i] * (1 - model.e[j]) for j in model.J)
    model.gender_count.add(model.m[i] == male_expr)

# Hotel room constraints
model.hotel_constraints = pyo.ConstraintList()
for i in model.I:
    # Female rooms - max 4 per room
    model.hotel_constraints.add(model.w[i] >= model.f[i] / 4)

    # Male rooms - max 4 per room
    model.hotel_constraints.add(model.y[i] >= model.m[i] / 4)

    # Total rooms
    model.hotel_constraints.add(model.z[i] == model.w[i] + model.y[i] + 1)


# Vehicle constraints
model.vehicle_constraints = pyo.ConstraintList()
for i in model.I:
    # Trucks for horses - each truck can carry 5 horses
    model.vehicle_constraints.add(sum(model.s[j, i] for j in model.J) <= 5 * model.o[i])

    # People transport - trucks can carry 2 people, traverses can carry 4
    model.vehicle_constraints.add(sum(model.s[j, i] for j in model.J) <= 2 * model.o[i] + 4 * model.n[i])

# Top k scores constraint
model.top_scores = pyo.ConstraintList()
for i in model.I:
    # Exactly k point riders per show
    model.top_scores.add(sum(model.b[j, i] for j in model.J) == model.k[i])

    # Can only be a point rider if attending
    for j in model.J:
        model.top_scores.add(model.b[j, i] <= model.s[j, i])

# Total cost constraint
gas_truck_cost = sum((model.a[i] / model.do) * model.g * model.o[i] for i in model.I)
gas_traverse_cost = sum((model.a[i] / model.dn) * model.g * model.n[i] for i in model.I)
rental_truck_cost = sum(model.ro * model.o[i] for i in model.I)
rental_traverse_cost = sum(model.rn * model.n[i] for i in model.I)
hotel_cost = sum(model.h[i] * model.z[i] * model.p[i] for i in model.I)
entry_cost = sum(model.q[i] * sum(model.s[j, i] for j in model.J) for i in model.I)
stall_cost = sum(model.t[i] * sum(model.s[j, i] for j in model.J) for i in model.I)

total_cost = gas_truck_cost + gas_traverse_cost + rental_truck_cost + rental_traverse_cost + hotel_cost + entry_cost + stall_cost
model.cost_constraint = pyo.Constraint(expr=total_cost <= model.c)


In [ ]:
# Solve the model
!apt-get install -y glpk-utils
import pyomo.environ as pyo
solver = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
result = solver.solve(model)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
# Print solution
print("\nOptimized Team Schedule")
print("=" * 50)
print(f"Total Team Score: {pyo.value(model.obj):.0f} points")
print(f"Total Cost: ${pyo.value(total_cost):.2f} of ${model.c} budget")
print("=" * 50)

# Show details for each show
for i in model.I:
    show_cost = (
        model.h[i] * pyo.value(model.z[i]) +
        model.q[i] * sum(pyo.value(model.s[j, i]) for j in model.J) +
        model.t[i] * sum(pyo.value(model.s[j, i]) for j in model.J) +
        (model.ro + (model.a[i] / model.do) * model.g) * pyo.value(model.o[i]) +
        (model.rn + (model.a[i] / model.dn) * model.g) * pyo.value(model.n[i])
    )

    print(f"\nShow: {show_names[i]}")
    print(f"  Total Attendees: {sum(pyo.value(model.s[j, i]) for j in model.J):.0f} of {model.l[i]} max")
    print(f"  Vehicles: {pyo.value(model.o[i]):.0f} trucks, {pyo.value(model.n[i]):.0f} traverses")
    print(f"  Hotel Rooms: {pyo.value(model.z[i]):.0f} total ({pyo.value(model.w[i]):.0f} female, {pyo.value(model.y[i]):.0f} male, 1 coach)")
    print(f"  Show Cost: ${show_cost:.2f}")

    show_score = 0
    print("\n  Team Members:")
    for j in model.J:
        if pyo.value(model.s[j, i]) > 0.5:  # Using 0.5 threshold due to potential floating point issues
            status = "Point Rider" if pyo.value(model.b[j, i]) > 0.5 else "Non-Point Rider"
            rider_score = model.x[i, j] if pyo.value(model.b[j, i]) > 0.5 else 0
            show_score += rider_score
            print(f"    {rider_names[j]}: {status}, Score: {rider_score if rider_score > 0 else 'N/A'}")

    print(f"\n  Show Team Score: {show_score:.0f}")

# Team member participation summary
print("\nTeam Member Participation Summary:")
for j in model.J:
    shows_attended = [show_names[i] for i in model.I if pyo.value(model.s[j, i]) > 0.5]
    point_shows = [show_names[i] for i in model.I if pyo.value(model.b[j, i]) > 0.5]
    total_points = sum(model.x[i, j] * pyo.value(model.b[j, i]) for i in model.I)

    print(f"\n{rider_names[j]}:")
    print(f"  Shows Attended: {len(shows_attended)} ({', '.join(shows_attended)})")
    print(f"  Point Rider At: {len(point_shows)} shows")
    print(f"  Total Points Contributed: {total_points:.0f}")


Optimized Team Schedule
Total Team Score: 5163 points
Total Cost: $16026.03 of $c budget

Show: SHTX Bryan
  Total Attendees: 5 of 6 max
  Vehicles: 1 trucks, 1 traverses
  Hotel Rooms: 3 total (2 female, 0 male, 1 coach)
  Show Cost: $2242.97

  Team Members:
    Jordan: Point Rider, Score: 289
    Justine: Point Rider, Score: 281
    Kate: Point Rider, Score: 291
    Carson: Non-Point Rider, Score: N/A
    Morgan: Point Rider, Score: 281

  Show Team Score: 1142

Show: SHTX World
  Total Attendees: 5 of 6 max
  Vehicles: 1 trucks, 1 traverses
  Hotel Rooms: 3 total (1 female, 1 male, 1 coach)
  Show Cost: $1970.11

  Team Members:
    Jordan: Point Rider, Score: 289
    Justine: Point Rider, Score: 281
    Kate: Point Rider, Score: 291
    Alex: Non-Point Rider, Score: N/A
    Morgan: Point Rider, Score: 281

  Show Team Score: 1142

Show: NRCHA
  Total Attendees: 3 of 3 max
  Vehicles: 2 trucks, 0 traverses
  Hotel Rooms: 2 total (1 female, 0 male, 1 coach)
  Show Cost: $2140.50

 